In [ ]:
# https://data.alaska.gov/Energy-and-Environment/Nenana-Ice-Classic-Ice-Breakup-Data/f2c3-azq8

In [22]:
import pandas as pd
import numpy as np
import math
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [23]:
nenana_ice_break_columns = ['Year', 'Date Time', 'Decimal Day of Year']
nenana_ice_break = pd.read_csv('datasets/nenana_ice_break.csv', usecols = nenana_ice_break_columns)
nenana_ice_break.sort_values('Decimal Day of Year')

,Year,Decimal Day of Year,Date Time
102,2019,104.0146,04/14/2019 12:21:00 AM
81,1998,110.7042,04/20/1998 04:54:00 PM
23,1940,111.6438,04/20/1940 03:27:00 PM
76,1993,113.5424,04/23/1993 01:01:00 PM
98,2015,114.6007,04/24/2015 02:25:00 PM
...,...,...,...
75,1992,135.2681,05/14/1992 06:26:00 AM
18,1935,135.5639,05/15/1935 01:32:00 PM
28,1945,136.4035,05/16/1945 09:41:00 AM
96,2013,140.6118,05/20/2013 02:41:00 PM


In [24]:
getMonth = lambda s : s[:2]
nenana_ice_break['month'] = nenana_ice_break['Date Time'].apply(getMonth)

In [25]:
nenana_ice_break

,Year,Decimal Day of Year,Date Time,month
0,1917,120.4792,04/30/1917 11:30:00 AM,04
1,1918,131.3979,05/11/1918 09:33:00 AM,05
2,1919,123.6063,05/03/1919 02:33:00 PM,05
3,1920,132.4486,05/11/1920 10:46:00 AM,05
4,1921,131.2792,05/11/1921 06:42:00 AM,05
...,...,...,...,...
99,2016,114.6521,04/23/2016 03:39:00 PM,04
100,2017,121.5000,05/01/2017 12:00:00 PM,05
101,2018,121.5542,05/01/2018 01:18:00 PM,05
102,2019,104.0146,04/14/2019 12:21:00 AM,04


In [26]:
nenana_ice_break['TOD'] = nenana_ice_break['Decimal Day of Year']%1
nenana_ice_break['TOY'] = nenana_ice_break['Decimal Day of Year'].apply(np.floor)

In [27]:
# time of day (normalized 0-1)
# ranges from 12:21AM - 11:23PM 
nenana_ice_break.sort_values('TOD')
    # we can see that 1991 has an entry error
    # the timestring does not match the Decimal Day of Year 
    # remove this year? 

,Year,Decimal Day of Year,Date Time,month,TOD,TOY
74,1991,121.0028,05/01/1991 12:04:00 PM,05,0.0028,121.0
102,2019,104.0146,04/14/2019 12:21:00 AM,04,0.0146,104.0
24,1941,123.0764,05/03/1941 01:50:00 AM,05,0.0764,123.0
6,1923,129.0833,05/09/1923 02:00:00 AM,05,0.0833,129.0
10,1927,133.2375,05/13/1927 05:42:00 AM,05,0.2375,133.0
...,...,...,...,...,...,...
69,1986,128.9514,05/08/1986 10:50:00 PM,05,0.9514,128.0
91,2008,127.9535,05/06/2008 10:53:00 PM,05,0.9535,127.0
77,1994,119.9590,04/29/1994 11:01:00 PM,04,0.9590,119.0
45,1962,132.9743,05/12/1962 11:23:00 PM,05,0.9743,132.0


In [28]:
# day number (out of 365)
# ranges from April 14 to May 20 
nenana_ice_break.sort_values('TOY') 

,Year,Decimal Day of Year,Date Time,month,TOD,TOY
102,2019,104.0146,04/14/2019 12:21:00 AM,04,0.0146,104.0
81,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
23,1940,111.6438,04/20/1940 03:27:00 PM,04,0.6438,111.0
76,1993,113.5424,04/23/1993 01:01:00 PM,04,0.5424,113.0
99,2016,114.6521,04/23/2016 03:39:00 PM,04,0.6521,114.0
...,...,...,...,...,...,...
75,1992,135.2681,05/14/1992 06:26:00 AM,05,0.2681,135.0
18,1935,135.5639,05/15/1935 01:32:00 PM,05,0.5639,135.0
28,1945,136.4035,05/16/1945 09:41:00 AM,05,0.4035,136.0
96,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0


In [29]:
nenana_ice_break.to_sql('nib', conn, if_exists='replace', index = False)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [30]:
nenana_ice_break.to_csv('labels.csv', index=False)

In [31]:
events = pd.read_sql("""
    SELECT * 
    from nib;
    """, con=conn)
events

,Year,Decimal Day of Year,Date Time,month,TOD,TOY
0,1917,120.4792,04/30/1917 11:30:00 AM,04,0.4792,120.0
1,1918,131.3979,05/11/1918 09:33:00 AM,05,0.3979,131.0
2,1919,123.6063,05/03/1919 02:33:00 PM,05,0.6063,123.0
3,1920,132.4486,05/11/1920 10:46:00 AM,05,0.4486,132.0
4,1921,131.2792,05/11/1921 06:42:00 AM,05,0.2792,131.0
...,...,...,...,...,...,...
99,2016,114.6521,04/23/2016 03:39:00 PM,04,0.6521,114.0
100,2017,121.5000,05/01/2017 12:00:00 PM,05,0.5000,121.0
101,2018,121.5542,05/01/2018 01:18:00 PM,05,0.5542,121.0
102,2019,104.0146,04/14/2019 12:21:00 AM,04,0.0146,104.0


In [32]:
events['month'].value_counts()

05    68
04    36
Name: month, dtype: int64